In [5]:
from numba import jit
import numpy as np
from numpy import *
from scipy.optimize import curve_fit, fmin
import scipy.special as sp

########### Curve fitting ################################################


def nss_curve(x, b1, b2, b3, b4, l1, l2):
    tl1 = x/l1
    tl2 = x/l2
    etl1 = exp(-tl1)
    etl2 = exp(-tl2)
    metl1 = (1 - etl1) / tl1
    metl2 = (1 - etl2) / tl2
    nss = b1 + b2 * metl1 + b3 * (metl1 - etl1) + b4 * (metl2 - etl2)
    return nss


def get_nss_params(x, y):
    nss_lim_lower = [0, -np.inf, -np.inf, -np.inf, 0.0001, 0.0001]
    nss_lim_upper = [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
    nss_var = curve_fit(nss_curve, x, y, bounds=(
        nss_lim_lower, nss_lim_upper), method='trf')[0]
    return nss_var


def get_nss_adv(x2, y2):
    x = array(x2)
    y = array(y2)
    def fp(c, x): return (c[0]) + (c[1]*((1 - exp(-x/c[4]))/(x/c[4]))) + (c[2]*((((1-exp(-x/c[4]))/(
        x/c[4]))) - (exp(-x/c[4])))) + (c[3]*((((1-exp(-x/c[5]))/(x/c[5]))) - (exp(-x/c[5]))))
    # error function to minimize
    def e(p, x, y): return ((fp(p, x)-y)**2).sum()
    # fitting the data with fmin
    # initial parameter value
    p0 = array([0.01, 0.01, 0.01, 0.01, 0.01, 1.00, 1.00])
    res = fmin(e, p0, args=(x, y), disp=False)
    return res


def get_regression_quality(lin_x, lin_y, x, y):
    total_points = len(x)
    dif = []
    for i in range(total_points):
        temp_y = np.interp(x[i], lin_x, lin_y)
        dif.append((y[i] - temp_y)**2)
    res = np.sum(dif)
    return res


def pyinterp(x, x_vals, y_vals):
    return np.interp(x, x_vals, y_vals)


def create(tenors, yields, new_issue_tenor):
    x = tenors
    y = yields
    x2 = []
    y2 = []
    x_max = x[-1]
    try:
        nss_params = get_nss_params(x, y)
    except:
        nss_params = get_nss_adv(x, y)
        nss_params = np.delete(nss_params, -1)

    nss_crv_x = np.linspace(x[0], x_max, 100)
    nss_crv_y = nss_curve(nss_crv_x, *nss_params)
    log_crv_func = np.poly1d(np.polyfit(log(x), y, 1))
    log_crv_x = nss_crv_x
    log_crv_y = log_crv_func(log_crv_x)

    nss_accuracy = get_regression_quality(nss_crv_x, nss_crv_y, x, y)
    log_accuracy = get_regression_quality(log_crv_x, log_crv_y, x, y)

    if nss_accuracy > log_accuracy:
        nss_crv_y = log_crv_y

    if x_max < new_issue_tenor and new_issue_tenor != '':
        x_min = nss_crv_x[90]
        for i in range(10):
            x2.append(nss_crv_x[90 + i])
            y2.append(nss_crv_y[90 + i])

        p = np.poly1d(np.polyfit(x2, y2, 3))
        crv_x2 = np.linspace(x2[-1] + 0.01, new_issue_tenor, 50)
        crv_y2 = p(crv_x2)
        nss_crv_x = np.concatenate((nss_crv_x, crv_x2), axis=None)
        nss_crv_y = np.concatenate((nss_crv_y, crv_y2), axis=None)

    return [nss_crv_x, nss_crv_y]


def nearest(x, a):
    if x >= max(a[0]):
        n = a[1][-1]
    elif x <= min(a[0]):
        n = a[1][0]
    else:
        c = int(len(a[0])/2)
        if x > a[0][c]:
            while a[0][c] <= x:
                c += 1
        else:
            while a[0][c] >= x:
                c -= 1
        n = a[1][c] if abs(a[0][c] - x) < abs(a[0][c - 1] - x) else a[1][c - 1]

    return n